In [1]:
import msticpy as mp

mp.init_notebook()
qry_prov = mp.QueryProvider("LocalData", data_paths=["./data"], query_paths=["./data"])


In [2]:
network_traffic_df = qry_prov.Network.get_network_flows()
network_traffic_df.head()


,Unnamed: 0,TenantId,TimeGenerated,FlowStartTime,FlowEndTime,FlowIntervalEndTime,FlowType,ResourceGroup,VMName,VMIPAddress,PublicIPs,SrcIP,DestIP,L4Protocol,L7Protocol,DestPort,FlowDirection,AllowedOutFlows,AllowedInFlows,DeniedInFlows,DeniedOutFlows,RemoteRegion,VMRegion,AllExtIPs,TotalAllowedFlows
0,881,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 14:22:40.697,2019-02-12 13:00:07.000,2019-02-12 13:45:08.000,2019-02-12 14:00:00.000,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,['65.55.44.109'],NaN,NaN,T,https,443.0,O,4.0,0.0,0.0,0.0,eastus2,eastus,65.55.44.109,4.0
1,877,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 14:22:40.681,2019-02-12 13:00:48.000,2019-02-12 13:58:33.000,2019-02-12 14:00:00.000,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,"['13.71.172.130', '13.71.172.128']",NaN,NaN,T,https,443.0,O,18.0,0.0,0.0,0.0,canadacentral,eastus,13.71.172.128,18.0
2,876,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 14:22:40.681,2019-02-12 13:00:48.000,2019-02-12 13:58:33.000,2019-02-12 14:00:00.000,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,"['13.71.172.130', '13.71.172.128']",NaN,NaN,T,https,443.0,O,18.0,0.0,0.0,0.0,canadacentral,eastus,13.71.172.130,18.0
3,879,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 14:22:40.681,2019-02-12 13:10:56.000,2019-02-12 13:55:57.000,2019-02-12 14:00:00.000,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,['40.124.45.19'],NaN,NaN,T,https,443.0,O,2.0,0.0,0.0,0.0,southcentralus,eastus,40.124.45.19,2.0
4,875,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 14:22:40.681,2019-02-12 13:22:20.000,2019-02-12 13:22:20.000,2019-02-12 14:00:00.000,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,['104.43.212.12'],NaN,NaN,T,https,443.0,O,1.0,0.0,0.0,0.0,centralus,eastus,104.43.212.12,1.0


In [3]:
ti = mp.TILookup()
ti_results = ti.lookup_iocs(
    data=network_traffic_df.head(100), obs_col="AllExtIPs", providers="OTX"
)
ti.browse_results(ti_results)


Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank


Observables processed: 100%|██████████| 100/100 [00:27<00:00,  3.60obs/s]


In [4]:
ip = ti_results[ti_results["Severity"] == "high"].iloc[0]["Ioc"]
network_traffic_df[network_traffic_df["AllExtIPs"] == ip].mp_plot.timeline_values(
    value_column="TotalAllowedFlows",
    title=f"Total allowed traffic to {ip}",
    kind="line",
    source_columns=["VMName", "FlowType"],
)


Loading BokehJS ...

Column(id='1076', ...)

In [5]:
from msticpy.analysis import timeseries

nt_decomp_df = network_traffic_df.mp_timeseries.analyze(
    time_column="TimeGenerated", data_column="TotalAllowedFlows", seasonal=7, period=24
)

nt_decomp_df.mp_timeseries.plot(
    y="TotalAllowedFlows",
)


Loading BokehJS ...

Column(id='1709', ...)

In [12]:
(  # Take our network traffic data
    network_traffic_df.iloc[35:55]
    # Lookup IPs at OTX
    .mp_pivot.run(IpAddress.ti.lookup_ip, providers="OTX", column="AllExtIPs")
    # Filter on high severity TI hits
    .query("Severity == 'high'")
    # Find where these IPs are geolocated
    .mp_pivot.run(IpAddress.geoloc, column="Ioc", join="left")
    # Plot these on a map
    .mp_plot.folium_map(ip_column="Ioc")
)


Observables processed: 100%|██████████| 20/20 [00:02<00:00,  7.76obs/s]
